In [1]:
import numpy as np
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pydantic as pd
from typing import Tuple

from data import FieldDataPlotly
import sys
sys.path.append('../../../')
from tidy3d import SimulationData

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.


In [2]:
class FieldDataComponent:
    
    def __init__(self, sim_data, monitor_name):

        self.monitor_name = monitor_name
        self.field_data = FieldDataPlotly(data=sim_data.monitor_data[monitor_name])
        self.field_val = list(self.field_data.data.data_dict.keys())[0]
        self.cs_axis = 2

        fields = list(self.field_data.data.data_dict.keys())
        freqs = self.field_data.data[self.field_val].f.values.tolist()
        cs_vals = self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()
        
        self.f_val = freqs[0]
        self.cs_val = np.mean(cs_vals)
        self.val_val = 'real'
    
    def plot_field(self):
        xyz_kwargs=dict(x=None, y=None, z=None)
        xyz_kwargs["xyz"[self.cs_axis]] = self.cs_val
        fig = self.field_data.plotly(
            field=self.field_val,
            freq=self.f_val,
            val=self.val_val,
            **xyz_kwargs
        )
        return fig
    
    def make_layout(self, app):
        
        def append_monitor_name(value):
            return f'{value}_{self.monitor_name}'

        layout = dcc.Tab(
            [
                html.Div(
                    [
                        html.Div(
                            [
                                dcc.Graph(
                                    id=append_monitor_name('field_plot'),
                                    figure=self.plot_field(),
                                )
                            ], style={'padding': 10, 'flex': 1}
                        ),
                        html.Div(
                            [
                                html.H1(f'Viewing data for FieldMonitor: {self.monitor_name}'),
                                html.H2(f'Field component.'),
                                dcc.Dropdown(
                                    options=list(self.field_data.data.data_dict.keys()),
                                    value=self.field_val,
                                    id=append_monitor_name('field_dropdown'),
                                ),
                                html.H2(f'Value to plot.'),
                                dcc.Dropdown(
                                    options=['real', 'imag', 'abs'],
                                    value=self.val_val,
                                    id=append_monitor_name('val_dropdown'),
                                ),            
                                html.Br(),
                                html.Div(
                                    [
                                        dcc.Dropdown(
                                            options=['x', 'y', 'z'],
                                            value='xyz'[self.cs_axis],
                                            id=append_monitor_name('cs_axis_dropdown'),
                                        ),
                                        dcc.Slider(
                                            min=self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()[0],
                                            max=self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()[-1],
                                            value=self.cs_val,
                                            id=append_monitor_name('cs_slider'),
                                        ),
                                    ],
                                )
                            ], style={'padding': 10, 'flex': 1}
                        ),
                    ], style={'display': 'flex', 'flex-direction': 'row'},
                )
            ],
            label=f'monitor: "{self.monitor_name}"'
        )

        @app.callback(
            Output(append_monitor_name('field_plot'), 'figure'),
            [
                Input(append_monitor_name('field_dropdown'), 'value'),
                Input(append_monitor_name('val_dropdown'), 'value'),
                Input(append_monitor_name('cs_axis_dropdown'), 'value'),
                Input(append_monitor_name('cs_slider'), 'value'),
                #Input('f_slider', 'value')
            ]
        )
        def set_field(value_field, value_val, value_cs_axis, value_cs):#, value_f):
            self.field_val = str(value_field)
            self.val_val = str(value_val)
            self.cs_axis = ['x', 'y', 'z'].index(value_cs_axis)
            self.cs_val = float(value_cs)
            
            fig = self.plot_field()
            return fig

        @app.callback(
            Output(append_monitor_name('cs_slider'), 'min'),
            Input(append_monitor_name('cs_axis_dropdown'), 'value'),
        )
        def set_min(value_cs_axis):#, value_f):
            self.cs_axis = ['x', 'y', 'z'].index(value_cs_axis)
            return self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()[0]

        @app.callback(
            Output(append_monitor_name('cs_slider'), 'max'),
            Input(append_monitor_name('cs_axis_dropdown'), 'value'),
        )
        def set_max(value_cs_axis):#, value_f):
            self.cs_axis = ['x', 'y', 'z'].index(value_cs_axis)
            return self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()[-1]
        
        return layout

In [3]:
class App:
    
    def __init__(self, sim_data : SimulationData):
        self.sim_data = sim_data
        self.monitor_data = [FieldDataComponent(self.sim_data, monitor_name) for monitor_name in self.sim_data.monitor_data.keys() if 'monitor' in monitor_name]
        self.app = self.make_app()

    def make_app(self):

        app = JupyterDash(__name__)
        app.layout = dcc.Tabs([data_component.make_layout(app) for data_component in self.monitor_data])
        return app

    def run(self):
    
        self.app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [4]:
sim_data = SimulationData.from_file('data/data_3_monitors.hdf5')
app = App(sim_data=sim_data)

In [5]:
app.run()